In [20]:
def set_hadoop_config(credentials):
    prefix = "fs.swift.service." + credentials['name'] 
    hconf = sc._jsc.hadoopConfiguration()
    hconf.set(prefix + ".auth.url", credentials['auth_url']+'/v3/auth/tokens')
    hconf.set(prefix + ".auth.endpoint.prefix", "endpoints")
    hconf.set(prefix + ".tenant", credentials['project_id'])
    hconf.set(prefix + ".username", credentials['user_id'])
    hconf.set(prefix + ".password", credentials['password'])
    hconf.setInt(prefix + ".http.port", 8080)
    hconf.set(prefix + ".region", credentials['region'])
    hconf.setBoolean(prefix + ".public", True)

In [21]:
auth_uri : 
global_account_auth_uri : 
username : 
password : 
auth_url : 
project : 
project_id : 
region : 
user_id : 
domain_id : 
domain_name : 
filename : 
container : 
tenantId : 


SyntaxError: invalid syntax (<ipython-input-21-2ace772ed7cd>, line 1)

In [22]:
year = '2007'

In [23]:
credentials = {
    'auth_url': '',
    'project_id': '',
    'region': '',
    'user_id': '',
    'username': '',
    'password': '',
    'filename': ',
    'container': ''
}
credentials_2 = {
    'auth_url': '',
    'project_id': '',
    'region': '',
    'user_id': '',
    'username': '',
    'password': '',
    'filename': '',
    'container': ''
}

In [24]:
credentials['name'] = 'crimes'
set_hadoop_config(credentials)

credentials_2['name'] = 'districts'
set_hadoop_config(credentials_2)

In [25]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

sc.addPyFile("https://raw.githubusercontent.com/seahboonsiew/pyspark-csv/master/pyspark_csv.py")
import pyspark_csv as pycsv


%matplotlib inline

import matplotlib.pyplot as plt
# matplotlib.patches allows us create colored patches, we can use for legends in plots
import matplotlib.patches as mpatches
# seaborn also builds on matplotlib and adds graphical features and new plot types

In [26]:
crimeData = sc.textFile("swift://notebooks." + credentials['name'] + "/" + credentials['filename'])
policeDeptDistrictData = sc.textFile("swift://notebooks." + credentials_2['name'] +"/" + credentials_2['filename'])

In [27]:
credentials['filename']

'2007data.csv'

In [28]:
crimeData_df = pycsv.csvToDataFrame(sqlContext, crimeData)


In [29]:
policeDeptDistrictData_df = pycsv.csvToDataFrame(sqlContext,policeDeptDistrictData)

In [30]:
crimeData_df.printSchema()

root
 |-- IncidntNum: integer (nullable = true)
 |-- Category: string (nullable = true)
 |-- Descript: string (nullable = true)
 |-- DayOfWeek: timestamp (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Time: timestamp (nullable = true)
 |-- PdDistrict: string (nullable = true)
 |-- Resolution: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- X: double (nullable = true)
 |-- Y: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- PdId: integer (nullable = true)



In [31]:
policeDeptDistrictData_df.printSchema()


root
 |-- COMPANY: string (nullable = true)
 |-- the_geom: string (nullable = true)
 |-- DISTRICT: string (nullable = true)
 |-- OBJECTID: integer (nullable = true)



In [32]:
policeDeptDistrictData_newCol_df = policeDeptDistrictData_df.selectExpr("COMPANY as COMPANY","the_geom as the_geom","DISTRICT as PdDistrict","OBJECTID as OBJECTID")
policeDeptDistrictData_newCol_df.printSchema()

root
 |-- COMPANY: string (nullable = true)
 |-- the_geom: string (nullable = true)
 |-- PdDistrict: string (nullable = true)
 |-- OBJECTID: integer (nullable = true)



In [33]:
crimeDataByDistrict = crimeData_df.filter(crimeData_df['PdDistrict'] != "null").groupBy("PdDistrict").count()
crimeDataByDistrictOrder = crimeDataByDistrict.sort("count",ascending=False)
for line in crimeDataByDistrictOrder.collect():
    print line

Row(PdDistrict=u'SOUTHERN', count=24175)
Row(PdDistrict=u'MISSION', count=18434)
Row(PdDistrict=u'NORTHERN', count=16430)
Row(PdDistrict=u'TENDERLOIN', count=14735)
Row(PdDistrict=u'BAYVIEW', count=13925)
Row(PdDistrict=u'CENTRAL', count=13335)
Row(PdDistrict=u'INGLESIDE', count=12797)
Row(PdDistrict=u'TARAVAL', count=9988)
Row(PdDistrict=u'PARK', count=7234)
Row(PdDistrict=u'RICHMOND', count=6586)


In [34]:
districtCrimeDataWithLongLat = crimeDataByDistrictOrder.join(policeDeptDistrictData_newCol_df,"PdDistrict")

In [35]:
districtCrimeDataWithLongLat.printSchema()

root
 |-- PdDistrict: string (nullable = true)
 |-- count: long (nullable = false)
 |-- COMPANY: string (nullable = true)
 |-- the_geom: string (nullable = true)
 |-- OBJECTID: integer (nullable = true)



In [36]:
districtCrimeDataWithLongLatJSON = districtCrimeDataWithLongLat.toJSON()

In [37]:
districtCrimeDataWithLongLatJSON.repartition(1).saveAsTextFile("swift://notebooks." + credentials['name'] + "/SFDistrictCrimesCount" + year + ".json")